In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import requests
from datetime import date, datetime, timedelta
from io import StringIO

from arable.client import ArableClient
from lib.physics import *
from lib.constants import *

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

In [ ]:
# grab some mark data
email = 'danielle@arable.com'
device = 'A000176' 
a = ArableClient()
a.connect(email, 'arable1', 'arable')

sta = "2017-07-04 08:00:00"
end = "2017-07-07 08:00:00"
#end = datetime.now()
#end = end.strftime("%Y-%m-%dT%H:%M:%SZ")


df = a.query(select='all', 
               format='csv', devices=[device], 
               measure="calibrated", 
               order="time", 
               end=end, start=sta, 
               limit=10000) 

df = StringIO(df)
df = pd.read_csv(df, sep=',', error_bad_lines=False)

# make sure I'm using a datetime object
df['time'] = pd.to_datetime(df['time'])
df.index = df['time']

df['RH'] = df['RH'] / 100.

In [ ]:
df['Tdew_leaf'] = Tdew_(df['Tbelow'], df['RH'])
df['Tdew_leaf_mk'] = df['Tdew_leaf']
df['Wet_leaf'] = df['Tdew_leaf'] >= df['Tbelow']*0.95
df['Tdew_air'] = Tdew_(df['Tair'], df['RH']/100)
df['Tdew_air_mk'] = Tdew_(df['Tdew_air'], df['RH'])
df['Wet_air'] = df['Tdew_air'] >= df['Tair']*0.95
mask = ~df['Wet_leaf']
df['Tdew_leaf_mk'].loc[mask] = np.nan

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12,9))
line1, = axes[0].plot(b08.index, df['Tdew_leaf'], color='#0067B5', label = 'Tdew', linewidth=1)
line2, = axes[0].plot(b08.index, df['Tbelow'], color='#bbbbbb', label = 'Tleaf')
line3, = axes[0].plot(b08.index, df['Tdew_leaf_mk'], color='#ff6768', label = 'Tdew', linewidth=3)
axes[0].set_ylabel('Temperature C')

line1, = axes[1].plot(df.index, df['Wet_leaf']*1, color='#ff6768', label = 'Tdew')
axes[1].set_ylim([-0.1, 1.1])
axes[1].set_ylabel('Wetness')

line1, = axes[2].plot(df.index, np.cumsum(df['Wet_leaf']/24), color='#ff6768', label = 'Tdew')
axes[2].set_ylabel('Wet Days')
